# Calculate virtual time series

In [19]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

# - - - - - - - - - - - - - - - - - - - - 
# DEFAULT IMPORTS - IN ALL NOTEBOKS
from src import configs

prjconf = configs.ProjectConfigParser()

# - - - - - - - - - - - - - - - - - - - - 
# NOTEBOOK SPECIFIC IMPORTS
import numpy as np
from pathlib import Path
import pandas as pd
import rasterio
from tqdm import tqdm

from eobox.raster import cube
from eobox.raster import gdalutils

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Define task

In [20]:
scene_collection_id = 1
scene_collection_name = f"scenecoll{scene_collection_id:02d}__tile-32UNU_year-2018_product-L30_maxcc-75_minsc-0"
variables = ["Red", "NIR", "SWIR1", "SWIR2"]
qa = "CLEAR"
qa_valid = [1]
chunksize = 2**9

scoll_layers = prjconf.get_layer_df_of_scene_collection(scene_collection_name, variables + [qa])
scoll_layers.head(6)
assert len(scoll_layers["tile"].unique()) == 1
assert len(scoll_layers["product"].unique()) == 1  
# here L30 & S30 would also make sense but then we need to change the product string below
tile = scoll_layers["tile"].unique()[0]
product = scoll_layers["product"].unique()[0]


idx_virtual = pd.date_range(start='2018-01-01', end="2018-12-31", freq='4W')
idx_virtual_strings = [date.strftime('%Y-%m-%d') for date in idx_virtual]
idx_virtual

dst_dir = prjconf.get_path("L3", "rasterdir") / tile / f"scoll{scene_collection_id:02d}"
dst_pattern = str(dst_dir) + "/" + f"{tile.lower()}__scoll{scene_collection_id:02d}__vts4w__" + "{date}__{var}.vrt"

print(idx_virtual)
print(dst_dir)
print(dst_pattern)

DatetimeIndex(['2018-01-07', '2018-02-04', '2018-03-04', '2018-04-01',
               '2018-04-29', '2018-05-27', '2018-06-24', '2018-07-22',
               '2018-08-19', '2018-09-16', '2018-10-14', '2018-11-11',
               '2018-12-09'],
              dtype='datetime64[ns]', freq='4W-SUN')
/home/ben/Devel/Projects/classify-hls/data/processed/L3/raster/T32UNU/scoll01
/home/ben/Devel/Projects/classify-hls/data/processed/L3/raster/T32UNU/scoll01/t32unu__scoll01__vts4w__{date}__{var}.vrt


## Run task

In [21]:
scoll = cube.EOCubeSceneCollection(df_layers=scoll_layers, 
                                   chunksize=chunksize, 
                                   variables=variables, 
                                   qa=qa, 
                                   qa_valid=qa_valid 
                                  )

In [22]:
scoll.create_virtual_time_series(
    idx_virtual=idx_virtual,
    dst_pattern=dst_pattern,
    dtypes="int16",
    compress='lzw',
    nodata=None,
    num_workers=8)
# 64it [2:24:38, 25.54s/it]

  0%|          | 0/4 [00:00<?, ?it/s]

Red: 64 / 64 chunks already processed and skipped.
NIR: 64 / 64 chunks already processed and skipped.
SWIR1: 64 / 64 chunks already processed and skipped.
SWIR2: 64 / 64 chunks already processed and skipped.


## Create VRTs

Create a time series layer stack (VRT) for each variable. 

In [23]:
dst_dir_vsts_stack = prjconf.get_path("L3", "rasterdir") / "VRTs" / "ts_per_band" / tile / f"scoll{scene_collection_id:02d}"
dst_dir_vsts_stack.mkdir(parents=True, exist_ok=True)
dst_dir_vsts_stack

PosixPath('/home/ben/Devel/Projects/classify-hls/data/processed/L3/raster/VRTs/ts_per_band/T32UNU/scoll01')

In [24]:
input_file_list
dst_dir

PosixPath('/home/ben/Devel/Projects/classify-hls/data/processed/L3/raster/T32UNU/scoll01')

In [26]:
for var in scoll.variables:
    input_file_list = list(list(Path(dst_dir).glob(f"*{var}*.vrt")))
    input_file_list = np.sort(input_file_list)
    output_file = Path(dst_dir_vsts_stack) / f"{tile.lower()}__scoll{scene_collection_id:02d}__vts4w__{var}.vrt"
    print(output_file)
    gdalutils.buildvrt(input_file_list, output_file, relative=True, separate=True)

/home/ben/Devel/Projects/classify-hls/data/processed/L3/raster/VRTs/ts_per_band/T32UNU/scoll01/t32unu__scoll01__vts4w__Red.vrt
/home/ben/Devel/Projects/classify-hls/data/processed/L3/raster/VRTs/ts_per_band/T32UNU/scoll01/t32unu__scoll01__vts4w__NIR.vrt
/home/ben/Devel/Projects/classify-hls/data/processed/L3/raster/VRTs/ts_per_band/T32UNU/scoll01/t32unu__scoll01__vts4w__SWIR1.vrt
/home/ben/Devel/Projects/classify-hls/data/processed/L3/raster/VRTs/ts_per_band/T32UNU/scoll01/t32unu__scoll01__vts4w__SWIR2.vrt
